In [36]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd
import numpy as np
import ta

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def trade_option(symbolID, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    #start_time = start_time.replace(hour=9, minute=15, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)

    ourl = 'https://groww.in/v1/api/stocks_fo_data/v4/charting_service/chart/exchange/BSE/segment/FNO/'+symbolID
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(ourl, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    ohlc['EMA_6'] = ohlc['close'].ewm(span=9, adjust=False).mean()
    return ohlc

In [38]:
symbol1 = 'BANKNIFTY2461950000CE'
symbol2 = 'NIFTY2462023300CE'
symbol3 = 'FINNIFTY2461822300CE'
days_before = 15
timeframe = 3
ohlc = trade_option(symbol2, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
daily_pl =0
daily_pl_limit = 10
canEnter = True
positive = 0
negative = 0
consecutive_losses = 0
curr_day = datetime.datetime.fromtimestamp(ohlc.iloc[0]['timestamp']).day
start_trade = False

for i, row in ohlc.iterrows():
    if i == 0:
        continue
    next_day = datetime.datetime.fromtimestamp(ohlc.iloc[i]['timestamp']).day
    
    # Reset can_enter_today for next day
    if curr_day < next_day:
        print("New Day")
        curr_day = next_day
        can_enter_today = False
        daily_pl = 0
        start_trade = False
        consecutive_losses = 0
        canEnter = True
        position = False
        entry_price = None
        
    # Check if trading time is after 9:40 AM
    if datetime.datetime.fromtimestamp(row['timestamp']).hour >= 9 and datetime.datetime.fromtimestamp(row['timestamp']).minute >= 45:
        start_trade = True

    if daily_pl <= -10 or daily_pl >= 15: #Max profit NIFTY 10 BANKNIFTY 15
        start_trade = False
    
    #run set max profit 20 and max loss 10 for day 
    if ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['EMA_6'] and ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['open'] and canEnter and start_trade and consecutive_losses < 2 :
        if not position:
          position = True
          entry_price = row['open'] #change to close at run time
          trades_taken += 1
          canEnter = False
          stop_loss = ohlc.iloc[i-1]['open']
          print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")
    
    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['EMA_6'] or row['open'] < stop_loss :
          position = False
          profit_loss = row['open'] - entry_price
          if profit_loss >0 :
              positive += 1
          if profit_loss <0 :
              negative += 1
              consecutive_losses += 1
          total_profit_loss += profit_loss
          daily_pl += profit_loss
          print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
          entry_price = None
    
    if position:
        stop_loss = max(stop_loss,row['open'])
    
    if position == False and canEnter == False and ohlc.iloc[i-1]['low'] < ohlc.iloc[i-1]['EMA_6']:
        canEnter = True

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nPercentage: {(positive/trades_taken)*100}%")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*15)-(trades_taken*60)}")

Long: Entry Time: 2024-06-13 13:23:00, Entry Price: 636.95
Exit Long: Time: 2024-06-13 13:47:00, Exit Price: 600.2, P/L: -36.75
New Day
Long: Entry Time: 2024-06-14 09:48:00, Entry Price: 473.95
Exit Long: Time: 2024-06-14 09:57:00, Exit Price: 500.0, P/L: 26.05000000000001

Trades count: 2 cost: 120

Percentage: 50.0%

Total Profit/Loss: -10.7

Take Away: -280.5
